In [156]:
import os
import time
from utils import batch_generate, tokens_generate, run_inference
import importlib
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from dotenv import load_dotenv
import plotly.express as px
import torch
import torch.nn.functional as F
import huggingface
from datetime import datetime
load_dotenv('secrets.env')

device = 'mps'
model_name = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'
seed = 88

In [112]:
# https://huggingface.co/datasets/causal-nlp/CLadder
# dataset = load_dataset('causalNLP/cladder')
# cladder = dataset['full_v1.5_default'].shuffle(seed)

# r1-distill qwen 1.5b (tokenizer + model) 
tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# set seed 
torch.manual_seed(seed)

KeyboardInterrupt: 

In [147]:
# tokens = tokens_generate(batches, tokenizer = tokenizer)
for batch_idx, batch in enumerate(tokens): 
    test = tokens[batch_idx]['tokenized_prompts']

In [208]:
outputs

[{'inference_time': 2.8605129718780518}]

# Experiment
Goal: test model performance on 100 cladder questions w/ chain of thought reasoning =D

In [226]:
importlib.reload(utils)
import utils
import time
import inspect 
# print(inspect.getsource(utils.run_inference))

In [214]:
cladder_df = pd.DataFrame(cladder[0:100])

batches = batch_generate(cladder_df)
tokens = tokens_generate(batches, tokenizer, device = 'mps')

        

    




Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


KeyboardInterrupt: 

In [64]:
%%time
prompt = "Hello, I/'d like to plan a birthday for my dog, can you help me?"

CPU times: user 5 μs, sys: 2 μs, total: 7 μs
Wall time: 16 μs


In [59]:
print(response)

Hello, I/'d like to plan a birthday for my dog, can you help me? I have a 2-year-old Golden Retriever, named Max. I want to know if I can do a dog birthday party, and if so, what are the steps I should follow. Also, I'm not sure about the details of the dog's birthday, so maybe I can just find out when that is. Additionally, I'm not sure how to organize this event. I need to find a way to make it fun and engaging for my dog. Any help would be greatly appreciated.
Alright, so I'm trying to plan a birthday for my golden retriever named Max. He's just two years old, so I need to figure out how to make that special day even more memorable. I've heard that birthdays for dogs can be a bit different from ours because dogs aren't humans, but I'm not entirely sure how that works. I also need to find out when Max's birthday is so I can plan accordingly. Let me break this down step by step.

First, I should figure out what Max's birthday is. I know that humans have birthdays every day, but dogs h